  # 🌟 Подробный туториал по ClearML для начинающих







  Привет! Этот туториал поможет тебе освоить **ClearML** — платформу для управления ML-экспериментами, данными, пайплайнами и отчётами.







  Мы пройдём всё **по шагам**:



  1. **Локальный запуск** (всё на твоём компьютере)



  2. **Запуск через ClearML Agent** (локально/на другом компьютере или в Google Colab)









  💡 **Требования**:



  - Установлен [`uv`](https://docs.astral.sh/uv/)



  - Базовые знания Python и ML



  - Аккаунт на [ClearML Web App](https://app.clear.ml/)







  📚 **Официальная документация**:



  - [ClearML Docs — Главная](https://clear.ml/docs/latest/docs/)



  - [Datasets (включая CLI `clearml-data`)](https://clear.ml/docs/latest/docs/clearml_data/)



  - [Agent (локальный и Colab)](https://clear.ml/docs/latest/docs/clearml_agent/)



  - [Google Colab + Agent](https://clear.ml/docs/latest/docs/guides/ide/google_colab/)



  - [Pipelines](https://clear.ml/docs/latest/docs/pipelines/)



  - [Reports (GUI)](https://clear.ml/docs/latest/docs/webapp/webapp_reports/)



  - [Logger](https://clear.ml/docs/latest/docs/fundamentals/logger/)







  🎥 [Официальный YouTube-плейлист](https://youtube.com/playlist?list=PLMdIlCuMqSTnoC45ME5_JnsJX0zWqDdlO)

  ## Установка и настройка

  Выполни в терминале:



  ```bash
  uv add clearml clearml-agent
  ```



 Но в данном репозитории данные пакеты уже добавлены, поэтому можешь просто синхронизировать через



 ```bash
  uv sync
  ```





  Затем:



  ```bash
  clearml-init
  ```







  Следуй инструкциям: получи API Key и Secret на [странице настроек](https://app.clear.ml/settings/workspace-configuration).







  **Где хранится конфиг?**



  - **Linux**: `~/clearml.conf`



  - **Mac**: `$HOME/clearml.conf`



  - **Windows**: `C:\Users\<твоё_имя>\.clearml\clearml.conf`

  ## Подготовка данных

  Создадим синтетический датасет для последующих экспериментов. Выполни эту ячейку:

In [1]:
from sklearn.datasets import make_classification
import pandas as pd
import os

# Создаём директорию data
os.makedirs('./data', exist_ok=True)

# Генерируем датасет
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)
df = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
df['target'] = y

# Сохраняем в CSV
df.to_csv('./data/synthetic_dataset.csv', index=False)

print("✅ Синтетический датасет создан и сохранён в ./data/synthetic_dataset.csv")



✅ Синтетический датасет создан и сохранён в ./data/synthetic_dataset.csv


  ## 1. Локальный режим: эксперимент с логгером







  Открой/запусти файл `1_task_with_logger.py`







  Этот пример демонстрирует простой эксперимент с полиномиальной регрессией, в котором:



  - Используется Pipeline с PolynomialFeatures и LogisticRegression



  - Обучается модель с разными степенями полинома



  - Логируются метрики точности, графики PCA, ROC-кривая и confusion matrix



  - Важно: параметр `output_uri=True` в `Task.init` крайне важен. При установке его в `False`



    файл модели .pkl не будет загружен на сервер ClearML, а будет сохранена только информация о модели



  - При вызове `joblib.dump` ClearML автоматически перехватывает и сохраняет модель на сервер







  Основные возможности логгера, которые демонстрируются в этом примере:



  - Логирование гиперпараметров с помощью `task.connect()`



  - Логирование метрик точности, precision, recall и F1-score с помощью `logger.report_scalar()`



  - Визуализация результатов с помощью `logger.report_matplotlib_figure()` и `logger.report_plotly()`



  - Логирование confusion matrix с помощью `logger.report_confusion_matrix()`



  - Сохранение таблиц с помощью `logger.report_table()`



  - Логирование финальных значений метрик с помощью `logger.report_single_value()`



  - Логирование текстовых сообщений с помощью `logger.report_text()`



  - Логирование примеров предсказаний для отладки

  ## 2. Запуск через ClearML Agent







  Теперь научимся запускать задачи **не на твоей машине, а на удалённой** (например, с GPU).







  Есть два варианта:



  - **Стандартный агент** (на твоём же или каком-нибудь еще компьютере с доступом в интернет)



  - **Google Colab** (предоставляет бесплатный GPU)

  ### Подготовка к запуску агента







  Важно понимать, что при запуске задач через агента, код будет выполняться не на вашей локальной машине, а на той машине, где запущен агент. Это означает, что:







  1. Все зависимости должны быть установлены на машине агента (благо ClearML делает это за вас в автоматическом режиме)



  2. Все файлы, необходимые для выполнения задачи, должны быть доступны агенту



  3. Если используется git, файл, который запускаем, должен быть включен в коммит, на котором вы находитесь (об этом ниже)









  ### Запуск агента локально







  1. Открой **новый терминал**



  2. Запусти агент:



     ```bash
     clearml-agent daemon --queue default
     ```



     Агент будет ждать задачи в очереди `default`.



     !!! Если запускать вне виртуальной среды, то используй команду:



     ```bash

     uvx clearml-agent daemon --queue default

     ```



  3. Запусти файл `2_remote_task.py` в **первом терминале**:



     ```bash
     python 2_remote_task.py
     ```



     Скрипт мгновенно завершится, но задача появится в очереди.



  4. Во **втором терминале** (где запущен агент) ты увидишь лог выполнения.







  🔗 Подробнее: [ClearML Agent Docs](https://clear.ml/docs/latest/docs/clearml_agent/)









  ### Запуск агента в Google Colab







  В Google Colab также можно запустить ClearML агента, но есть некоторые особенности:







  1. **Команда запуска агента**: В официальной документации ClearML есть готовый [Colab-ноутбук](https://colab.research.google.com/github/clearml/clearml/blob/master/examples/clearml_agent/clearml_colab_agent.ipynb) для запуска агента, но в нем используется устаревшая команда. Для избежания проблем с matplotlib используйте команду:



     ```bash
     !env MPLBACKEND=tkagg clearml-agent daemon --queue default
     ```



     Это устанавливает правильный бэкенд для matplotlib, чтобы избежать ошибок при отображении графиков.







  2. **Использование GPU**: Если ваша задача требует GPU, перед запуском агента измените тип среды выполнения:



     - Перейдите в меню "Runtime" → "Change runtime type"



     - Выберите "Hardware accelerator" → "GPU"









  ### Почему мы генерируем датасет в этом примере?







  В файле `2_remote_task.py` вы можете заметить, что датасет генерируется непосредственно в коде (строка 63: `X, y = make_classification(...)`). Это делается потому, что:







  1. Агент не имеет доступа к локальным файлам на вашей машине



  2. Если бы мы использовали локальный файл датасета, агент не смог бы его прочитать



  3. Генерация датасета в коде гарантирует, что все необходимые данные будут доступны при выполнении на агенте







  В реальных проектах для работы с датасетами рекомендуется использовать ClearML Datasets (как показано в пункте 3), которые позволяют безопасно передавать использовать датасеты на агентах.







  ### Практика: Запуск задачи с локальным датасетом на агенте







  Предлагаем выполнить следующее упражнение:







  1. Возьмите задачу из пункта 1 "Локальный режим: эксперимент с логгером" (файл `1_task_with_logger.py`)



  2. Отправьте эту задачу на выполнение агенту через GUI ClearML. [Видеоинструкция](https://www.youtube.com/watch?v=MX3BrXnaULs&list=PLMdIlCuMqSTnoC45ME5_JnsJX0zWqDdlO&index=5)



  3. Наблюдайте ошибку отсутствия датасета, так как агент не имеет доступа к локальным файлам







  Это продемонстрирует важность использования ClearML Datasets для обеспечения доступности данных на агентах.







  ### Важные замечания при использовании git







  Если вы используете git в вашем проекте, для успешного запуска задач на агенте необходимо соблюдать следующие условия:







  1. **Файл в коммите**: Файл, который вы запускаете (в данном случае `2_remote_task.py`), должен быть включен в коммит, на котором вы находитесь. Агент скачивает репозиторий, и поэтому код, который вы хотите выполнить, должен быть закоммичен.







  2. **Совпадение коммитов**: Коммит, на котором вы находитесь на вашем локальном компьютере, должен совпадать с коммитом на сервере, где запущен агент. Это важно, потому что агент копирует код с вашего репозитория, и если коммиты будут отличаться, могут возникнуть проблемы с выполнением задачи.







  Это особенно важно для начинающих: агент работает с репозиторием, поэтому убедитесь, что ваш код закоммичен и версии совпадают на локальной машине и на сервере.

## 3. Работа с датасетом ClearML Dataset

В предыдущих примерах мы столкнулись с проблемой: агент не имеет доступа к локальным файлам. ClearML Dataset решает эту проблему, предоставляя централизованное хранилище для датасетов с версионированием.

### Преимущества ClearML Datasets:
- **Версионирование**: автоматическое отслеживание изменений в данных
- **Доступность**: датасеты доступны из любого места (локально, на агенте, в Colab)
- **Кэширование**: данные кэшируются локально для быстрого доступа
- **Воспроизводимость**: каждая версия датасета имеет уникальный ID
- **Метаданные**: можно добавлять описание, теги и статистику

### Два способа работы с датасетами:

1. **CLI-инструмент `clearml-data`** - удобен для создания и управления датасетами из командной строки
2. **Python API** - программный доступ к датасетам из кода

---

### 3.1 Создание датасета через CLI

Создадим датасет из нашего локального файла `synthetic_dataset.csv`:

```bash
# Создаем новый датасет
clearml-data create --project Tutorial --name "Synthetic Dataset"

# Добавляем файлы в датасет
clearml-data add --files ./data/synthetic_dataset.csv

# Загружаем датасет на сервер и финализируем
clearml-data close
```

После выполнения `clearml-data close` датасет будет загружен на сервер и готов к использованию!

**Дополнительные команды:**

```bash
# Просмотр списка датасетов
clearml-data list --project Tutorial

# Получение информации о датасете по имени и проекту
clearml-data get --project Tutorial --name "Synthetic Dataset"

# Или по ID (если известен)
clearml-data get --id <dataset_id>

# Скачивание датасета локально по имени
clearml-data get --project Tutorial --name "Synthetic Dataset" --copy ./downloaded_data

# Или по ID
clearml-data get --id <dataset_id> --copy ./downloaded_data
```

💡 **Важно**: Работа по имени/проекту автоматически использует последнюю версию датасета, что очень удобно для разработки!

---

### 3.2 Создание датасета через Python API

Выполни файл `3_dataset_creation.py` - он создаст датасет программно и загрузит его на сервер ClearML.

Основные возможности:
- Создание датасета из локальных файлов
- Добавление метаданных и статистики
- Версионирование датасетов
- Автоматическая загрузка на сервер

---

### 3.3 Использование датасета в эксперименте

Выполни файл `4_task_with_dataset.py` - это улучшенная версия первого примера, которая:
- **Автоматически загружает датасет по имени и проекту** (без необходимости указывать ID!)
- Работает как локально, так и на агенте
- Автоматически кэширует данные для повторного использования
- Отслеживает зависимость эксперимента от версии датасета
- Всегда использует последнюю версию датасета (если не указан конкретный ID)

Теперь эту задачу можно безопасно запустить на агенте через GUI ClearML, и датасет будет автоматически загружен!

---

### 3.4 Версионирование датасетов

ClearML автоматически создает новую версию датасета при изменении данных. Работа по имени/проекту **всегда использует последнюю версию**, что удобно для разработки:

```python
# Создаем новую версию, указав родительский датасет
from clearml import Dataset

new_dataset = Dataset.create(
    dataset_project="Tutorial",
    dataset_name="Synthetic Dataset",  # То же имя!
    parent_datasets=["<parent_dataset_id>"]  # ID предыдущей версии
)
new_dataset.add_files(path="./data/updated_dataset.csv")
new_dataset.upload()
new_dataset.finalize()

# Теперь Dataset.get() по имени автоматически вернёт новую версию!
```

**Преимущества версионирования:**
- 📈 Отслеживание эволюции данных
- 🔄 Сравнение результатов экспериментов на разных версиях
- ↩️ Откат к предыдущим версиям (указав конкретный ID)
- 🎯 Для production: зафиксируй конкретный ID версии для воспроизводимости
- 🛠️ Для разработки: используй имя/проект для автоматического обновления

---

### 3.5 Практическое задание

1. Создайте датасет через CLI из файла `synthetic_dataset.csv` (или запустите `3_dataset_creation.py`)
2. Запустите `4_task_with_dataset.py` локально - он автоматически найдёт датасет по имени
3. Отправьте эту же задачу на выполнение агенту через GUI
4. Убедитесь, что задача успешно выполняется на агенте с доступом к датасету

💡 **Совет**: Используйте работу по имени/проекту для разработки - это удобнее, чем каждый раз искать ID. Для production можно зафиксировать конкретный ID версии датасета.

---

### 3.6 Шпаргалка по работе с датасетами

**CLI команды:**
```bash
# Создать датасет
clearml-data create --project MyProject --name "My Dataset"

# Добавить файлы
clearml-data add --files ./data/file.csv

# Загрузить и финализировать
clearml-data close

# Просмотр списка
clearml-data list --project MyProject

# Информация о датасете
clearml-data get --project MyProject --name "My Dataset"

# Скачать локально
clearml-data get --project MyProject --name "My Dataset" --copy ./output
```

**Python API:**
```python
from clearml import Dataset

# Создание датасета
dataset = Dataset.create(
    dataset_project="MyProject",
    dataset_name="My Dataset"
)
dataset.add_files(path="./data")
dataset.upload()
dataset.finalize()

# Загрузка последней версии (РЕКОМЕНДУЕТСЯ)
dataset = Dataset.get(
    dataset_project="MyProject",
    dataset_name="My Dataset"
)

# Загрузка конкретной версии
dataset = Dataset.get(dataset_id="specific_id_here")

# Получить путь к данным
data_path = dataset.get_local_copy()

# Список файлов
files = dataset.list_files()
```

📚 **Документация**: 
- [ClearML Datasets SDK](https://clear.ml/docs/latest/docs/clearml_data/clearml_data_sdk)
- [ClearML Data CLI](https://clear.ml/docs/latest/docs/clearml_data/clearml_data_cli)